In [9]:
import torch
import sys
sys.path.append('../..')

from torchuq.transform.distcal_discrete import *
from torchuq.evaluate.distribution_cal import *
from torchuq.dataset.classification import *
from sklearn.linear_model import LogisticRegression
from torchuq.evaluate import categorical
from matplotlib import pyplot as plt

In [10]:


uci_datasets = ['digits', 'adult', 'iris', 'breast-cancer']
subset_uci = ['digits']


In [7]:
for name in subset_uci:
	# 60% Train, 20% Calibration, 20% Test dataset
	dataset = get_classification_datasets(name, val_fraction=0.2, test_fraction=0.2, split_seed=0, normalize=True, verbose=True)
	
	train_dataset, cal_dataset, test_dataset = dataset
	X_train, y_train = train_dataset[:][0], train_dataset[:][1]
	X_cal, y_cal = cal_dataset[:][0], cal_dataset[:][1]
	X_test, y_test = test_dataset[:][0], test_dataset[:][1]
	
	# Simple logistic regression classifier trained
	reg = LogisticRegression(random_state=0).fit(X_train, y_train)
	print(f"Classification accuracy on Train: {reg.score(X_train, y_train):.2}")
	print(f"Classification accuracy on Test: {reg.score(X_test, y_test):.2}")
	


	# Predict probabilistic outcome on K classes, on the calibration and test datasets 
	pred_cal = torch.Tensor(reg.predict_proba(X_cal.numpy()))

	pred_test = torch.Tensor(reg.predict_proba(X_test.numpy()))

	

	# Use the DiscreteDistCalibrator class and train it on the calibration dataset

	calibrator = DiscreteDistCalibrator(verbose=True)
	calibrator.train(pred_cal, torch.Tensor(y_cal))

	output_cal = calibrator(pred_cal)
	output_test = calibrator(pred_test)

	# Evaluation
	
	print(f"[Calibration Dataset] Log loss before calibration = {discrete_cal_loss(y_cal, pred_cal)}; After calibration={discrete_cal_loss(y_cal, output_cal)}")
	print(f"[Calibration Dataset] Calibration score before calibration = {discrete_cal_score(y_cal, pred_cal)}, After calibration = {discrete_cal_score(y_cal, output_cal)}")

	print(f"[Test Dataset] Log loss before calibration = {discrete_cal_loss(y_test, output_test)}; After calibration={discrete_cal_loss(y_test, output_test)}")
	print(f"[Test Dataset] Calibration score before calibration = {discrete_cal_score(y_test, output_test)}, After calibration = {discrete_cal_score(y_test, output_test)}")
	

Loading dataset adult....
Splitting into train/val/test with 19537/6512/6512 samples
Done loading dataset adult
LR Train + test scores 0.8459845421507908 0.8538083538083538
[Cal] comparing log loss 0.3388117049331117 0.35355412679534826
[Cal] comparing calibration plots 0.2615702801851912 0.24089886084725295
[Test] comparing log loss 0.3209573961365611 0.33681698013805067
[Test] comparing calibration plots 0.251880504359195 0.22676487763709735
Loading dataset iris....
Splitting into train/val/test with 90/30/30 samples
Done loading dataset iris
LR Train + test scores 0.9777777777777777 0.9
[Cal] comparing log loss 0.18161406212796768 0.12677776548009867
[Cal] comparing calibration plots 0.19774074074074077 0.11759259259259258
[Test] comparing log loss 0.3159977881237864 0.28814987395304326
[Test] comparing calibration plots 0.19459259259259262 0.1372962962962963
Loading dataset breast-cancer....
Splitting into train/val/test with 341/114/114 samples
Done loading dataset breast-cancer
L